In [33]:
# Top 50000 words from glove.42B.300d.zip were written to top_50000.txt

# Load libraries
import numpy as np
import pandas as pd
from scipy import spatial
from random import sample
import copy
import re
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Load all data from massive embeddings file. You only need to do this once
embeddings_all = {}
with open("./top_50000.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        tag = nltk.pos_tag([word])[0][1]
        if tag in ['NN', 'NNP'] and len(word) != 1:
            vector = np.asarray(values[1:], "float32")
            embeddings_all[word] = vector

# Load list of codenames words.
codenames_words = []
with open("./codenames_words.txt", 'r') as f:
    for line in f:
        codenames_words.append(line.strip())

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/prastutisingh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/prastutisingh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [36]:
# Choose smaller sets of embeddings. 
def embeddings_size(embeddings_size, embeddings_all):
    #embeddings_size is the length of the subset of embeddings
    #embeddings_all is the embeddings from 50k top words
    embeddings = {}
    for x in list(embeddings_all)[0:embeddings_size]:
        embeddings[x] = embeddings_all[x]
    
    #adds words from codenames that don't make it in the top XX words
    for item in codenames_words:
        if item not in embeddings:
            try: 
                embeddings[item] = embeddings_all[item]
            except KeyError: 
                codenames_words.remove(item)
            
    return embeddings

In [37]:
#embeddings_1000 = embeddings_size(1000, embeddings_all)
#embeddings_5000 = embeddings_size(5000, embeddings_all)
embeddings_10000 = embeddings_size(10000, embeddings_all)
embeddings_25000 = embeddings_size(25000, embeddings_all)

In [38]:
# Helper Functions
def distance(embeddings, word, reference):
    return spatial.distance.cosine(embeddings[word], embeddings[reference])

def closest_words(embeddings, reference):
    return sorted(embeddings.keys(), key=lambda w: distance(embeddings, w, reference))

def goodness(embeddings, word, answers, bad):
#     if word in answers + bad: return -999
    return sum([distance(embeddings, word, b) for b in bad]) - 4.0 * sum([distance(embeddings, word, a) for a in answers])

def minimax(embeddings, word, answers, bad):
#     if word in answers + bad: return -999
    return min([distance(embeddings, word, b) for b in bad]) - max([distance(embeddings, word, a) for a in answers])

def candidates(embeddings, answers, bad, size=20):
    best = sorted(embeddings.keys(), key=lambda w: -1 * goodness(embeddings, w, answers, bad))
    res = [(str(i + 1), "{0:.2f}".format(minimax(embeddings, w, answers, bad)), w) for i, w in enumerate(sorted(best[:250], key=lambda w: -1 * minimax(embeddings, w, answers, bad))[:size])]
    return [c[2] for c in res]

In [39]:
answers = ["iron", "ham", "beijing"]
bad = ["fall", "witch", "note", "cat", "bear", "ambulance"]
candidates(embeddings_all, answers, bad, size=20)

['tong',
 'wok',
 'guan',
 'kitchenware',
 'nippon',
 'torino',
 'thanh',
 'jian',
 'bao',
 'jia',
 'omelette',
 'kan',
 'quan',
 'sheng',
 'ge',
 'savoy',
 'lyon',
 'shu',
 'intercontinental',
 'buffet']

In [ ]:
# Functions to help with word + scores (not necessary for data collection)...
from itertools import zip_longest

def grouper(n, iterable, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

def tabulate(data):
    data = list(grouper(10, data))
    return data

In [51]:
from random import sample
from itertools import combinations 

def generate_single_example(codenames_words, embeddings):
    # Generates a single example (good words, bad words, top ten clues)
    while True: 
        good = sample(codenames_words, 3)
        combo_list = list(combinations(good, 2)) 
        threshold_list = [distance(embeddings_all, item[0], item[1]) > 0.75 for item in combo_list]
        if all(good): 
            break
    
    bad = sample(codenames_words, 6)
    
#     for i in range(len(bad)): 
#         while bad[i] in good: 
#             bad[i] = sample(codenames_words, 1)
    
    print(good)
    print(bad)
    clues = candidates(embeddings, good, bad)
    
    for item in good: 
        if item in clues:
            clues.remove(item)

    return good, bad, clues[:10]

def write_n_examples(codenames_words, embeddings, n, filename): 
    # Generates n examples and writes them to filename
    output_file = open(filename, 'w')
    
    for i in range(n): 
        good, bad, clues = generate_single_example(codenames_words, embeddings)
        word_string = ', '.join(good + bad + clues) + '\n'
        output_file.write(str(i) + '.' + word_string)
    
    output_file.close()
    
# write_n_examples(codenames_words, embeddings_1000, 10, 'test_examples.txt')

In [48]:
distance(embeddings_all, 'ham', 'iron')

0.7916841804981232

In [ ]:
def read_n_examples(filename):
    '''Reads filename of good, bad, clues examples into a three-dimensional array. 
    To access the 0th example: data[0]
    To access the good words (in list form) from the 1st example: data[1][0]
        good words list - 0
        bad words list - 1
        clue words list - 2
    To access the first clue word from the 1st example: data[1][2][0]'''
    file = open(filename, 'r')
    
    data = []

    for line in file: 
        full_list = line.strip('\n').split(',')
        good = full_list[:3]
        bad = full_list[3:9]
        clues = full_list[9:]
        data.append([good, bad, clues])
        
    file.close()
    return data

In [52]:
#write_n_examples(codenames_words, embeddings_10000, 10, 'examples_10000_vocab.txt')
write_n_examples(codenames_words, embeddings_25000, 10, 'examples_25000_vocab.txt')

['hood', 'kid', 'rome']
['bow', 'truck', 'death', 'box', 'worm', 'bond']
['undertaker', 'death', 'club']
['kangaroo', 'orange', 'tower', 'bark', 'phoenix', 'shot']
['wall', 'fish', 'pilot']
['triangle', 'doctor', 'nurse', 'dog', 'millionaire', 'tower']
['lemon', 'opera', 'lap']
['sprint', 'hospital', 'belt', 'alien', 'mammoth', 'time']
['triangle', 'mug', 'lock']
['mouth', 'telescope', 'berry', 'belt', 'box', 'africa']
['truck', 'kangaroo', 'opera']
['rock', 'lock', 'staff', 'eye', 'ivory', 'revolution']
['slip', 'whip', 'mercury']
['dwarf', 'bark', 'note', 'lab', 'slip', 'root']
['box', 'africa', 'novel']
['knife', 'himalaya', 'kiwi', 'lap', 'lab', 'doctor']
['europe', 'triangle', 'time']
['bank', 'cell', 'giant', 'antarctica', 'stick', 'sprint']
['fish', 'lap', 'worm']
['lock', 'horseshoe', 'knight', 'night', 'jam', 'turkey']


In [ ]:
write_n_examples(codenames_words, embeddings_5000, 100, 'examples_5000_vocab.txt')
write_n_examples(codenames_words, embeddings_10000, 100, 'examples_10000_vocab.txt')